In [343]:
import numpy as np
import matplotlib.pyplot as plt
from utils.extract_gps import extract_gps
import cv2
from scipy.spatial.transform import Rotation, ScipyRotation
from pyproj import Proj, Transformer
import math

ImportError: cannot import name 'ScipyRotation' from 'scipy.spatial.transform' (/home/zeyadcode/Videos/RobEn/.venv/lib/python3.10/site-packages/scipy/spatial/transform/__init__.py)

In [344]:
def haversine(lat1, lon1, lat2, lon2, R=6371000):
    """
    Calculate the great-circle distance between two points
    on the Earth using the haversine formula.

    Parameters:
      lat1, lon1: latitude and longitude of point 1 (in degrees)
      lat2, lon2: latitude and longitude of point 2 (in degrees)
      R: Earth radius in meters (default 6,371,000 m)

    Returns:
      Distance between the two points in meters.
    """
    # convert degrees to radians
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    Δφ = math.radians(lat2 - lat1)
    Δλ = math.radians(lon2 - lon1)

    # haversine formula
    a = math.sin(Δφ / 2) ** 2 + math.cos(φ1) * math.cos(φ2) * math.sin(Δλ / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c


# Example with your points:
lat_pred, lon_pred = 29.81833793, 30.82473345
lat_true, lon_true = 29.81891470, 30.82603970

error_m = haversine(lat_pred, lon_pred, lat_true, lon_true)
print(f"Prediction error: {error_m:.2f} meters")

Prediction error: 141.40 meters


In [507]:
data_pnts = [
    {
        "frame_number": 2364,
        "pixel_pos": [1065, 2009],
    },
    {
        "frame_number": 5573,
        "pixel_pos": [3683, 1249],
    },
    {
        "frame_number": 3675,
        "pixel_pos": [3443.11, 2107.63],
    },
    
]

data_pnt = data_pnts[1]


# R and t matrix preparation
video_path = "./data/GX011072.MP4"
frame_number = data_pnt["frame_number"]
fps = 30

gps_data = extract_gps(video_path, frame_number, fps)
gps_data

Loading GoPro Data Track: 0 [  0.0 B]  [  0.0 s/B]
Loading GoPro Data Track: 3,790,072 [  3.6 MiB]  [333.7 MiB/s]
    Timer(GPMD - Called: 1, Total: 0.28309, Avg: 0.28309, Rate: 3.53)
>> Found GPS9 
    Timer(extract GPS - Called: 1, Total: 0.62280, Avg: 0.62280, Rate: 1.61)
    Timer(extract ACCL - Called: 1, Total: 3.32022, Avg: 3.32022, Rate: 0.30)
    Timer(extract GRAV - Called: 1, Total: 1.33047, Avg: 1.33047, Rate: 0.75)
    Timer(extract CORI - Called: 1, Total: 1.83382, Avg: 1.83382, Rate: 0.55)
Timer(parsing - Called: 1, Total: 7.39102, Avg: 7.39102, Rate: 0.14)


{'lat': 29.8193836,
 'lon': 30.8260162,
 'alt': 297.087 <Unit('meter')>,
 'yaw': -0.4699841058365676 <Unit('radian')>,
 'pitch': 0.5529265994253082 <Unit('radian')>,
 'roll': 0.9355096358215894 <Unit('radian')>}

In [ ]:
# WGS84 system
camera_lat = gps_data['lat']
camera_lon = gps_data['lon']
camera_alt = gps_data['alt'].magnitude

yaw = gps_data['yaw'].magnitude
pitch = gps_data['pitch'].magnitude
roll = gps_data['roll'].magnitude

GROUND_ALT = 269 # 246 is the correct, 309.4 worked on frame 5573 # ! THE FLAG ALTITUDE POSITION, LIKE LITERAL ALTITUDE POSITION, THIS IS NOT RELATIVE TO ANYTHING

#  6oct: 246
# cam: 232.81

In [509]:
calibration_result = np.load('./calibration_result.npz')

pixel_pos = np.array(data_pnt['pixel_pos'], dtype=np.float32)
pixel_pos_reshaped = np.array([[pixel_pos]]) # 1x1x2
K = calibration_result['K'] # 3x3
K[2, 2] = 1.0

K_inv = np.linalg.inv(K)
dist_coeffs = calibration_result['dist_coeffs'] # 1x5

print(K, K.shape)
print(dist_coeffs, dist_coeffs.shape)

[[4.66270808e+03 0.00000000e+00 2.68948998e+03]
 [0.00000000e+00 4.66111981e+03 2.48375385e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] (3, 3)
[[ 1.57115562e-01 -1.80126890e+00  1.56035942e-03 -7.29443881e-03
   9.98523709e+00]] (1, 5)


In [510]:
R_cam2imu = np.array(
    [
        [1, 0, 0],  # IMU‑X =  1·Cam‑X + 0·Cam‑Y + 0·Cam‑Z
        [0, 0, 1],  # IMU‑Y =            1·Cam‑Z
        [0, -1, 0],  # IMU‑Z = -1·Cam‑Y
    ]
)

R_cam2body = np.array(
    [
        [0, -1, 0],  # body X = -cam Y
        [1, 0, 0],  # body Y =  cam X
        [0, 0, 1],  # body Z =  cam Z
    ]
)

In [ ]:
# distortation and normalization
undistorted_point = cv2.undistortPoints(pixel_pos, K, dist_coeffs, None, K)
ray_cam = np.array([undistorted_point[0][0][0], undistorted_point[0][0][1], 1.0])

# Convert from normalized image plane coordinates to a proper 3D direction vector
ray_cam = np.linalg.inv(K) @ ray_cam
ray_cam = ray_cam / np.linalg.norm(ray_cam)  # Normalize to unit vector

# Convert optical -> IMU
ray_cam = R_cam2imu @ ray_cam

# Rotation from ENU world frame to body frame
R_world_to_body = ScipyRotation.from_euler("zyx", [yaw, pitch, roll], degrees=False)
R_cam_to_world = R_world_to_body.inv()

# 5. Transform ray into the World Frame
ray_world = R_cam_to_world.apply(ray_cam)

# 6. Ray-Plane Intersection
camera_pos_enu = np.array([0, 0, camera_alt])

In [512]:
print(ray_world)
if ray_world[2] >= 0 and False:
    print("Error: Ray is not pointing towards the ground. Check pitch and roll angles.")
else:
    t = (GROUND_ALT - camera_pos_enu[2]) / ray_world[2]
    intersection_enu = camera_pos_enu + t * ray_world

    # 7. Convert ENU intersection point back to Lat/Lon
    # Define the WGS84 and local ENU projections
    wgs84 = "EPSG:4326"  # Lat/Lon
    enu_proj = f"+proj=aeqd +lat_0={camera_lat} +lon_0={camera_lon} +ellps=WGS84"

    # Create the transformer
    transformer = Transformer.from_crs(enu_proj, wgs84, always_xy=True)

    # Transform the East (X) and North (Y) coordinates
    lon, lat = transformer.transform(intersection_enu[0], intersection_enu[1])

    print("\n--- Calculated Ground Coordinates ---")
    print(f"Latitude:  {lat:.8f}")
    print(f"Longitude: {lon:.8f}")

correct_lat = 29.8189147
correct_lon = 30.8260397

error = haversine(lat, lon, correct_lat, correct_lon)
error

[-0.87033819 -0.08254568  0.48548701]

--- Calculated Ground Coordinates ---
Latitude:  29.81942668
Longitude: 30.82653711


74.45619641458195